# Extensions needed in this Project

For the Implementation of Additive Phylogeny the following librarys are needed, because they contain important extensions.  

In [1]:
// #r @"C:\Users\Vanesssa\source\repos\BioFSharp\src\BioFSharp\bin\Release\netstandard2.0\BioFSharp.dll" 
// #r @"C:\Users\Vanesssa\source\repos\BioFSharp\src\BioFSharp.IO\bin\Release\netstandard2.0\BioFSharp.IO.dll"
// #r @"C:\Users\Vanesssa\source\repos\FSharp.Stats\src\FSharp.Stats\bin\Release\netstandard2.0\FSharp.Stats.dll

#r "nuget: BioFSharp, 2.0.0-preview.3"
#r "nuget: BioFSharp.IO, 2.0.0-preview.3"
#r "nuget: FSharp.Stats, 0.5.0"
#r"nuget: FSharp.Data, 6.2.0"

open FSharp.Data
open BioFSharp
open BioFSharp.IO
open FSharp.Stats
open System.Collections.Generic

Installed Packages BioFSharp, 2.0.0-preview.3 BioFSharp.IO, 2.0.0-preview.3 FSharp.Stats, 0.5.0

# Additive Phylogeny


Additive Phylogeny is a distance based Approach to construct phylogenetic trees from an additive distance matrix. A distancematrix is the representation of nodes (species) and edges (distance between species) as table and is additive,if you can recieve a unique simple unrooted tree fitting the distance matrix. In Real World Problems additive matrices are often not present, but parts of analysis are very similar to non-additive matrices. For this reason the implementation of the method additive Phylogeny is an important step in understanding computing evolutionary phylogenetic trees and solving special cases of real world problems.<br>
One of the central key points to recieve a additive phylogenetic tree is to  compute the limb length of an leaf j to its parent. (j represents the index of the species from interest) Since all matrixes have to be symmetric you can simply iter over it by using the rowindex, being equal to the corresponding columnindex. 
First you have to create a type Distancematrix for simulating the Matrix; consisting of a discription of the row and the column of matrix and the matrix itsself.

In [2]:
// Create Type DistanceMatrix with members species and DistanceM for representation of a distancematrix, connecting the different species by distances

type DistanceMatrix<'a> = { 
    Species : string [] 
    DistanceM : matrix
    } with 
        static member Create species distanceM = {Species=species; DistanceM=distanceM} 

In [3]:
// create testmatrix from Type Distancematrix, equal to matrix from the book "Bioinformatics Algorithm Chapter 7"

let myMatrixAdditive<'a> = 
    let distTest = 
        [| 
            [|0.;13.;21.;22.|] 
            [|13.;0.;12.;13.|] 
            [|21.;12.;0.;13.|]
            [|22.;13.;13.;0.|]
        |] 
        |> matrix
        
    DistanceMatrix<'a>.Create [|"i";"j";"k";"l"|] distTest

# The Limb Length Problem

<b>The first step</b> to get the corresponding phylogenetic tree of an additive distance matrix is to create a function Limblenthformula that computes the distance of a distinct leaf j to its parent leaf. Leaves are always present-day species. The formula needed for this function is:<br>
<b>Limblength(j) = (D(i,j)+D(j,k)-D(i,k))/2 </b> <br> where  i,j,k are three different species.

One further point you have to consider is that the specindex must be present in the matrix and not equal or bigger than the length of the Matrix, so you should introduce a error warning, by an if-clause, that automatically gives an error if the specIndex is not part of the matrix.


In [4]:
// Formula for Limblength : Limblength(j) = D(i,j)+D(j,k)-D(i,k)/2
// specIndex 1 : present day species you want to know j , species 2 and 3: other Species in the tree  i and k 
// specCount: size of distanceMatrix --> number of species

let LimblengthFormula (distMat:matrix) (specIndex1:int) (specIndex2:int) (specIndex3:int) = 
    let specCount = distMat.NumRows
    if (specIndex1 >= specCount) || specIndex2 >=specCount || specIndex3 >= specCount then 
        failwithf "specIndex is not present in distance matrix" 
     
    (distMat.[specIndex2,specIndex1] + distMat.[specIndex1, specIndex3] - distMat.[specIndex2, specIndex3]) / 2. 

In [5]:
// Test of function
LimblengthFormula(myMatrixAdditive.DistanceM) 1  0 2 // 13 + 12 -21 / 2 = 2

2

Because we want to get and compare all Limblength of a given species we have to use a for-loop which collect all Limblength for all possible Combinations of species. But you have to consider that the specIndexes have to be different, because the third node is a point to compare and is used as a reference. To get every possible interaction only once, because of the symmetry, you have to think about that the first index is always smaller than the 2nd. Furthermore choose always the smallest one limblength of a distinct species which represents the limblength of this species for the actual distancematrix

In [6]:
// Determine all possible Limblength of given Species without taking a combination of species double --> can be done because symmetrie of matrix
// if the condition is fullfilled, you have to compute the Limblength stored in an Array. Last you only want the minima, 
// because this is the Limblength of the species for the matrix

let LimbLength (distMat: matrix) (specIndex: int) = 
    let specCount = distMat.NumRows
   
    [|for specIndex2 = 0 to (specCount-1) do 
        for specIndex3 = 0 to (specCount-1) do
            if (specIndex3 > specIndex2) && (specIndex3) <> (specIndex) && (specIndex) <> specIndex2 then 
                (LimblengthFormula (distMat) (specIndex) (specIndex2) (specIndex3))
    |]
    |>Array.min 

In [7]:
// test of Limblength
LimbLength myMatrixAdditive.DistanceM 3
LimbLength twentyNineMatrix.DistanceM 28

Error: input.fsx (3,12)-(3,28) typecheck error Der Wert, Namespace, Typ oder das Modul "twentyNineMatrix" ist nicht definiert.

# Trimming the tree

When we know the Limblength of any node in Tree(D), we can construct Tree (D) recursively using an algorithm described in the Chapter 7 of the Book "Bioinformatics Algorithms: An Active Learning Approach". <br>
<img src="method.png" alt="drawing" width="15%"/> <br>
For this, you need an additive matrix representing Tree(D) and pick an arbitrary leaf j. The distances of leaf j to other present day specioes is then updated by reducing thr current distance in the corresponding column / row through the Limblength of j and create matrix D<sub>bald</sub>. D<sub>bald</sub> is the distancematrix, where the length of limb(j) is zero, because every part in the corresponding row /column is reduced by the limblength. This D<sub>bald</sub> matrix gives you informations about the length of edges between the species and where the Attachment point has to be. The Attachment of i and k to l is for example af the distance Dbald in line four of this picture and is an essential step during creation of matrix

In [8]:
// Distance (species,j) - min Limblength(j)
// Construct the Dbald matrix where the row and column of the species from interest is modified by subtracting the value by the minimal Limblength

let getDbaldMatrixFormula (distMat:matrix)  (specIndexRow: int) (specIndexColumn: int) (specIndex: int)  =  
    if  distMat.[specIndexRow,specIndexColumn] = 0. then 
        0. 
    else
        distMat.[specIndexRow,specIndexColumn] - LimbLength (distMat) (specIndex)

In [9]:
// Test of function
getDbaldMatrixFormula (myMatrixAdditive.DistanceM) (1) (0) (1)

11

This function is used to change the row and column cooresponding to the species of this Limblength. For this a new matrix is build and filled with the old and the new values 

In [10]:
//Connstruction of the Dbaldmatrix for every Value of the columnb / row

let dBaldMatrix (distMat:matrix) (specIndex: int)   = 
     let specCountRow = distMat.NumRows
     let specCountColumn = distMat.NumCols

     [|for x = 0 to specCountRow-1 do 
          [|for y = 0 to specCountColumn-1 do
               if x = specIndex || y = specIndex then 
                    getDbaldMatrixFormula distMat x y specIndex             
               else              
                    distMat.[x,y]            
          |]
     |]
     |> matrix     

In [11]:
// Test of function dBaldMatrix
printfn "%A" (dBaldMatrix myMatrixAdditive.DistanceM 1 )


           0      1      2      3
                                 
 0 ->  0.000 11.000 21.000 22.000
 1 -> 11.000  0.000 10.000 11.000
 2 -> 21.000 10.000  0.000 13.000
 3 -> 22.000 11.000 13.000  0.000

Matrix of 4 rows x 4 columns


The next step is removal of changed column and rows from the dBald matrix to get matrix D<sup>trimmed</sup> to get closer to the neighbor. For this the last edge / species has to be removed. The Construction of dBald and Dtrimmed is repeated until a length of 2*2 matrix is reached 

In [12]:
// Trimming the row and column you have determined the Attached Point before to get a smaller distance matrix
let delete_species (distMat:matrix) (specIndex: int)  =
        distMat
        |>Matrix.removeColAt specIndex
        |> Matrix.removeRowAt specIndex      

In [ ]:
// test of function
delete_species (dBaldMatrix (myMatrixAdditive.DistanceM) 1) 1

[ 0, 21, 22, 21, 0, 13, 22, 13, 0 ] Item 3 3matrix [[0;21;22];
[21;0;13];
[22;13;0]]
 ElementOps FSharp.Stats.Instances+FloatNumerics@116 NumRows 3 NumCols 3 IsDense True IsSparse False InternalSparseColumnValues System.InvalidOperationException: not a sparse matrix\r\n at FSharp.Stats.Matrix`1.get_InternalSparseColumnValues() in C:\Users\bvenn\source\repos\FSharp.Stats\src\FSharp.Stats\AlgTypes.fs:line 152\r\n at lambda_method107(Closure, DenseRepr)\r\n at Microsoft.DotNet.Interactive.Formatting.Membe... TargetSite Int32[] get_InternalSparseColumnValues() Name get_InternalSparseColumnValues DeclaringType FSharp.Stats.Matrix<T> ReflectedType FSharp.Stats.Matrix<T> MemberType Method MetadataToken 100663519 Module FSharp.Stats.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ModuleVersionId a6db3a87-bbd2-4333-294d-e59bb078f351 MetadataToken 1 ScopeName FSharp.Stats.dll Name FSharp.Stats.dll Assembly FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Users/Vanesssa/.nuget/packages/fsharp.stats/0.5.0/lib/netstandard2.0/FSharp.Stats.dll FullName FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 FSharp.Stats.IFsiFormattable 1 FSharp.Stats.IMatrixFormattable 2 FSharp.Stats.IVectorFormattable 3 FSharp.Stats.IRowVectorFormattable 4 FSharp.Stats.Interval`1[a] 5 FSharp.Stats.Interval`1+Tags[a] 6 FSharp.Stats.Interval`1+Closed[a] 7 FSharp.Stats.Interval`1+LeftOpen[a] 8 FSharp.Stats.Interval`1+RightOpen[a] 9 FSharp.Stats.Interval`1+Open[a] 10 FSharp.Stats.Interval`1+Closed@DebugTypeProxy[a] 11 FSharp.Stats.Interval`1+LeftOpen@DebugTypeProxy[a] 12 FSharp.Stats.Interval`1+RightOpen@DebugTypeProxy[a] 13 FSharp.Stats.Interval`1+Open@DebugTypeProxy[a] 14 FSharp.Stats.BigRationalLarge 15 FSharp.Stats.BigRational 16 FSharp.Stats.BigRational+Tags 17 FSharp.Stats.BigRational+Z 18 FSharp.Stats.BigRational+Q 19 FSharp.Stats.BigRational+Z@DebugTypeProxy (2084 more) IsCollectible False ManifestModule FSharp.Stats.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ModuleVersionId a6db3a87-bbd2-4333-294d-e59bb078f351 MetadataToken 1 ScopeName FSharp.Stats.dll Name FSharp.Stats.dll Assembly FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null ModuleHandle System.ModuleHandle CustomAttributes [ ] ReflectionOnly False Location C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 FSharp.Stats.IFsiFormattable 1 FSharp.Stats.IMatrixFormattable 2 FSharp.Stats.IVectorFormattable 3 FSharp.Stats.IRowVectorFormattable 4 FSharp.Stats.Interval`1[a] 5 FSharp.Stats.Interval`1+Tags[a] 6 FSharp.Stats.Interval`1+Closed[a] 7 FSharp.Stats.Interval`1+LeftOpen[a] 8 FSharp.Stats.Interval`1+RightOpen[a] 9 FSharp.Stats.Interval`1+Open[a] 10 FSharp.Stats.BigRationalLarge 11 FSharp.Stats.BigRational 12 FSharp.Stats.BigRational+Tags 13 FSharp.Stats.BigRational+Z 14 FSharp.Stats.BigRational+Q 15 FSharp.Stats.INumeric`1[T] 16 FSharp.Stats.IIntegral`1[T] 17 FSharp.Stats.IFractional`1[T] 18 FSharp.Stats.IFloating`1[T] 19 FSharp.Stats.IIEEE`1[T] (400 more) IsFullyTrusted True CustomAttributes index value 0 [Microsoft.FSharp.Core.FSharpInterfaceDataVersionAttribute((Int32)2, (Int32)0, (Int32)0)] 1 [System.Runtime.Versioning.TargetFrameworkAttribute(".NETStandard,Version=v2.0", FrameworkDisplayName = "")] 2 [System.Reflection.AssemblyCompanyAttribute("Benedikt Venn, Timo Muehlhaus, Heinrich Lukas Weil, David Zimmer, Kevin Schneider, fslab open source contributors")] 3 [System.Reflection.AssemblyConfigurationAttribute("Release")] 4 [System.Reflection.AssemblyDescriptionAttribute("F#-first linear algebra, machine learning, fitting, signal processing, and statisti

# Finding the Attachment Point

Another necessary function is attachmentPoint, where you try to identify the point ot attachment for a special leaf. For this especially the dBald matrix is necessary and the corresponding distances.
"To find the attachment point of a leaf j in TREE(Dtrimmed), consider TREE(Dbald), which is the same as TREE(D) except that LIMBLENGTH(j) = 0. From the Limb Length Theorem, we know that there must be leaves i and k in TREE(Dbald) such that (Dbald<sub>i,j</sub> + Dbald<sub>j,k</sub> - Dbald<sub>i,k</sub>) / 2 = 0 
This leads to the assumption that the attachment point for leaf j must be located at distance Dbald <sub>i,j</sub> from leaf i on the path connecting i and k in the trimmed tree. This attachment point may occur at an existing node, in which case we connect j to this node. On the other hand, the attachment point for j may occur along an edge, in which case we place a new node at the attachment point and connect j to it." This formula is also used in similar style at the recursive function in the end

Limblength : Dbald (i,k) = Dbald (i,j) + Dbald(j,k)   

In [ ]:
// Determine the Attachmentpoint for one species when it fullfills the defined conditions

let attachmentPoint (d_trimmed: matrix) (endpoint: int) =
    let specCount = d_trimmed.NumRows-1

    [|for i = 0 to specCount-1 do 
        for j = i+1 to specCount-1 do
            if d_trimmed.[i,j] = (d_trimmed.[i,endpoint] + d_trimmed.[endpoint,j]) then 
                   (i,j),d_trimmed.[i,(endpoint)]
    |]
    |> Array.groupBy (fun ((i,j),d) -> i)
    |> Array.maxBy fst
    |> snd
    |> Array.head
    

In [ ]:
attachmentPoint (dBaldMatrix myMatrixAdditive.DistanceM 3) 3

((1, 2), 6) Item1 (1, 2) Item1 1 Item2 2 Item2 6

In [ ]:
((attachmentPoint ( dBaldMatrix twentyNineMatrix.DistanceM 28)) 28)


((26, 27), 868) Item1 (26, 27) Item1 26 Item2 27 Item2 868

## The four point condition

The four point condition is a simple way to determine if a matrix is additive, instead of looking if you can visit every node only once (p.52). It explains that a matrix is only additive, when the four point condition holds for every quartet of indices of this matrix. Because the following algorithm only works for additive matrices, you should test that before. The formula needed therefore is:

d<sub>i,j</sub> + d<sub>k,l</sub> < = d<sub>i,k</sub> + d<sub>j,l</sub> = d<sub>i,l</sub> + d<sub>j,k</sub>

In [ ]:
//  four-point condition, condition needed to be additive

let fourpoint_condition (distMat: matrix) (specIndex0:int) (specIndex1: int) (specIndex2: int) (specIndex3: int) = 
    let specCount = distMat.NumRows
    if (specIndex0 >= specCount) || specIndex1 >=specCount ||specIndex2 >= specCount ||specIndex3 >= specCount then 
        failwithf "specIndex is not present in distance matrix"
    
    let sum1 = distMat.[specIndex0,specIndex1] + distMat.[specIndex2,specIndex3]
    let sum2 = distMat.[specIndex0,specIndex2] + distMat.[specIndex1,specIndex3]
    let sum3 = distMat.[specIndex0,specIndex3] + distMat.[specIndex1,specIndex2]

    (sum1 = sum2 && sum1 > sum3) || (sum1 = sum3 && sum1 > sum2) || sum2 = sum3 && sum2 > sum1

The dynamic version of testing Additivity looks as first condition, if we have a symmetric matrix, because this is necessary. Than you proof for every possible 4 points in the matrix if it fullfills the four point condition if not you should fail. To have every possible combination only once, you take care that the specIndex can always not be the same species

In [ ]:
// test if the matrix is additive for every possible four points in matrix
let testingAdditivity (distMat: matrix)   = 
    let specCount = distMat.NumRows
    let rowlength = distMat.NumRows
    let columnlength = distMat.NumCols

    if rowlength <> columnlength then failwith " Matrix isnt symmetric, but has to be to have a phylogenetic tree"
    
    for specIndex0 = 0 to (specCount-1) do 
            for specIndex1 = 0 to (specCount-1) do
                for specIndex2 = 0 to (specCount-1) do 
                    for specIndex3 = 0 to (specCount-1) do
                            if  
                                specIndex0 < specIndex1 
                                && specIndex0 < specIndex2 
                                && specIndex0 < specIndex3
                                && specIndex1 < specIndex2 
                                && specIndex1 < specIndex3
                                && specIndex2 < specIndex3
                            then 
                                if not (fourpoint_condition distMat specIndex0 specIndex1 specIndex2 specIndex3) then 
                                    failwithf "Matrix is non additive, but it has to be additive to fit a tree"

In [ ]:
// test matrix of book for additivity
fourpoint_condition myMatrixAdditive.DistanceM 0 1 2 3
testingAdditivity myMatrixAdditive.DistanceM

Here we test a 29*29 matrix for additivity to proof if the algorithm also works for bigger matrices

In [ ]:
let twentyNineMatrix<'a> =
    let test =
        let rawData = Http.RequestString @"https://bioinformaticsalgorithms.com/data/extradatasets/evolution/Additive_Phylogeny.txt"
        rawData.Split '\n'
        |> Array.skip 2
        |> Array.take 29
        |> Array.map (fun x ->
            x.Split ' '
            |> Array.take 29
            |> Array.map float    
        )  
        |> matrix

    DistanceMatrix<'a>.Create [|"i";"j";"k";"l";"m";"n";"o";"p";"q";"r";"s";"t";"u";"v";"w";"x";"y";"z";"a";"b";"c";"d";"e";"f";"g";"h";"ii";"jj";"kk"|] test

In [ ]:
// Testing of Additivity from 29*29 matrix
testingAdditivity twentyNineMatrix.DistanceM

Testing of the algorithm with a nonadditive matrix

In [ ]:
// Create Non Additive Matrix for Testing

let myNonAdditiveAdditive<'a> = 
    let distTest = 
        [| 
            [|0.;3.;4.;3.|] 
            [|3.;0.;4.;5.|] 
            [|4.;4.;0.;2.|]
            [|3.;5.;2.;0.|]
        |] 
        |> matrix
        
    DistanceMatrix<'a>.Create [|"i";"j";"k";"l"|] distTest

In [ ]:
// Test of an Non Additive Matrix 
fourpoint_condition myNonAdditiveAdditive.DistanceM 0 1 2 3
testingAdditivity myNonAdditiveAdditive.DistanceM 

Error: System.Exception: Matrix is non additive, but it has to be additive to fit a tree
   at Microsoft.FSharp.Core.PrintfModule.PrintFormatToStringThenFail@1448.Invoke(String message)
   at FSI_0023.testingAdditivity(Matrix`1 distMat)
   at <StartupCode$FSI_0028>.$FSI_0028.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

##  An algorithm for distance-based phylogeny construction

After constructing all essential functions for the additive phylogeny, we can "engineer" a recursive algorithm <b> AdditivePhylogeny </b>, for finding a simple tree fitting the n*n additive distance matrix D. 
For the implemented function Limb (D,j), computing Limblebgth(j) you have now to consider that you use the last row in D instead of selecting a arbitrary leaf j from the Tree(D). As template for the recursive Algorithm,you can use the scheme on p.18

In [ ]:
type PhylTreeConnection = { 
    SourceToTargetIndex: (int*int)
    Distance: float
    } with 
        static member Create sourceToTargetIndex distance = { SourceToTargetIndex= sourceToTargetIndex; Distance=distance} 

The steps to get succesfull the corresponding phylogenetic tree of a additive distancematrix are described in the following key points:
- testing of distanceMatrix: Call the function testingAdditivity for the original Matrix for testing if matrix is additive and cancel program if matrix is not additive
- define needed variables of Algorithm
- Start Algorithm describing the recursive function with original matrix and n discribing the size / number of colummns (remember in informatics, we always start with index 0)
    - When we just have two columns (n = 1), then we can easily determine the phylogenetic tree, because the distance between both species is known, so we can add that to the list resultlist and follwing conditions are skiped
    - if we have a bigger matrix, then we need to "analyze" the matrix in some steps:
        - put conditions in a while loop, which ends when n smaller one to reduce it until n smaller two
        - while loop computes for every matrix first the corresponding limnblength for the last column and updates then the distancematrix dist_mat by using the d-Bald function
            - Store for every distance matrix the d_Bald Matrix as well as size and limblength in a mutable dictionary graph
            - furthermore we store for every matrix the attachment points representing degenerate triples which satisfies Di,j = Di,n-1 + Dj,n-1 and store them in a 2nd mutable dictionary. This attachment points are needed to add possible 
            - in the end we update n, because n is needed to be equal to the size of matrix 
        - if n is now smaller two, than we add last the two* two matrix to the dictionary manually and take 0 as limblength, now we are able to reconstruct the phylogenetic tree. For zjid we also define path as the distance between i and j connected to each other
        - Next we need to integrate new nodes, if v is not already part of the path

In [ ]:
let addPhyl (distMat:matrix) =
    (* Hier wird getestet ob die in die Funktion gegebene distanzmatrix die Anforderungen einer additiven Matrix erfüllt, 
    da der Algorithmus nur für eine additive Matrix funktionsfähig ist. Dafür wird die Funktion testingAdditivity aufgerufen.
    *)
    testingAdditivity distMat

    // Hier werden verschiedene Variablen (zum Teil veränderbar) deklariert. Diese werden im Laufe des Algorithmus benötigt und müssen zum Teil veränderbar sein

    // veränderbare Variable, welche die aktuelle Version der Matrix beinhaltet, da die Matrix verändert werden muss (Stichwort Dbald)
    let mutable dist_Mat = distMat

    // veränderliche Varialble, die immer auf die letzte Reihe / Spalte der aktuellen Matrix verweist  
    let mutable n = dist_Mat.NumRows-1 

    // Unveränderliche Variable, welche auf die Länge der InputMatrix verweist
    let lengthOfMatrix = distMat.NumRows-1

    // Hier wird eine leere Liste vom Typ PhylTreeConnection deklariert in der die Pfade am Ende gelistet sind und der Tree beschrieben wird 
    let mutable resultList: PhylTreeConnection list = []

    (*  Hier werden twei mutable dictionaries deklariert. graph ist dafür notwendig, alle Matrixen zu speichern und damit rückwärts auflösen zu können  
        und enthält den Wert von n als Schlüssel ebensp wie der attachmentsstore der src,target und den attachmentPoint speichert. 
    *)
    let graph = new Dictionary<int,(float*matrix) >()
    let attachmentsStore = new Dictionary<int,((int*int)*float)>()

    // internalNodes ist die variable, welche den Index des einzufügenden Knoten deklariert
    let mutable internalNodes = 2*dist_Mat.NumRows-3
    
    
    (* Hier wird geprüft, ob eine 2*2 Matrix vorliegt, oder eine größere Matrix.
        Wenn eine 2*2 Matrix vorliegt, dann muss der Algorithmus nicht weiter durchlaufen werden 
        und es kann direkt der phylogenetische Baum, bestehend aus einer einfachen Verbindung der beiden Spezies erstellt werden.
        Die Länge der Verbindung entspricht der Distanz zwischen den beiden Spezies. Folglich wird  der Baum in der Liste 
        dargestellt durch den Eintrag von Source to Target (0,1) bzw. (1,0) und der Distanz zwischen beiden Spezies und die Liste wird zurückgegeben   
    *)
    if n = 1 then
        let distance = dist_Mat.[0,1]
        resultList <- {SourceToTargetIndex = (0,1); Distance = distance}::resultList
        resultList <- {SourceToTargetIndex = (1,0); Distance = distance}::resultList
        resultList

    // Diese Verzweigung wird ausgeführt falls keine 2*2 Matrix vorliegt
    
    else

    (*  
        Hier werden die in Zeile 24 und 25 deklarieren Dictionaries graph und attachmentsStore solange gefüllt bis eine Matrixgröße 2+2 erreicht ist.
        Das Dictionary graph beinhaltet dabei als Schlüssel n und ordnet diesem die jeweilige limbLength sowie die Distanzmatrix zu aus welcher die aktuelle Pfad Länge entnommen werden kann
        Im Dictionary attachmentsStore werden die berechneten attachmentpoints von zwei Spezies gespeichert sowie src und target
    *)
        while n >= 2 do
            let limblength = LimbLength (dist_Mat) n
            dist_Mat <- dBaldMatrix dist_Mat n 
            graph.Add (n,(limblength,dist_Mat))

       
           
             (*  printfn "n: %i" n
                printfn "limblength:  %f %A" limblength dist_Mat
                Beispielhafte Ausgabe wenn n = 3 erreicht ist, zur Ansicht was in dictionary graph gespeichert wird
             
             n: 3
             limblength: 409.000000 
                    0        1        2        3
                                                
            0 ->    0.000 3036.000 4777.000 1132.000
            1 -> 3036.000    0.000 6323.000 2678.000
            2 -> 4777.000 6323.000    0.000 3645.000
            3 -> 1132.000 2678.000 3645.000    0.000
             *)

            let attachment = attachmentPoint dist_Mat n
            let x = snd attachment 
            let (src,target) = fst attachment
            
            attachmentsStore.Add(n,((src,target),x))
            (* printfn " n: %i x: %f src: %i target:%i" n x src target 
            Beispielshafte Ausgabe wenn n = 3 erreicht ist in der 29*29 Matrix aus Rosalind
            n: 3 x: 2678.000000 src: 1 target:2
            *)
                
            // Hier wird die Matrix getrimmt und n geupdatet sodass n wieder dem Index der letzten Reihe / Spalte entspricht

            dist_Mat <- delete_species dist_Mat n
            n <- n-1
        
        (*
        Bis hier wurde die distanzmatrix iterativ reduziert und die jeweiligen Attachmentpoints und limblength in graph und attachmentsStore gespeichert
        Nun muss im TraceBack-Schritt in jeder Iteration ein initial graph bnestehen aus node 0 und 1 mit allen fehleden nodes vervollständigt werden. Diese werden 
        an den punkten aus attachmentstore angefügt mit der limblength, die in graph gespeichert ist. 
        Dafür wird zunächst der aktuelle Pfad der in der Matrix als Schnittpunkt 0,1 dargestellt ist in resultList eingefügt 
        *)

        resultList <- {SourceToTargetIndex = (0,1); Distance = distMat.[0,1]}::resultList
        resultList <- {SourceToTargetIndex = (1,0); Distance = distMat.[1,0]}::resultList

        // Hier wird n wieder angepasst sodass n = 2 ist und damit dem kleinsten Schlüssel in den Dictionaries entspricht
        n <- n+1

        (*  
            Im folgenden Schritt wird der traceback durchgeführt und nach und nach die bestimmten attachmentpoints eingefügt. 
            Dafür wird eine zweite While loop erstellt die solange wie n nicht dem maximalen Specindex entspricht den Traceback durchführt.
        *)
        let traceback =
            while n < lengthOfMatrix do (

            
                (*Hier wird die aktuelle resultList für jeden Durchgang geprintet:  printfn "resultList: %A" resultList, Bsp. zu Beginn und nach dem ersten Durchlauf  
                resultList zu Beginn: [{ SourceToTargetIndex = (1, 0) Distance = 3036.0 }; { SourceToTargetIndex = (0, 1) Distance = 3036.0 }]
                resultList nach Durchgang 1:[{ SourceToTargetIndex = (0, 55) Distance = 745.0 }; { SourceToTargetIndex = (55, 0) Distance = 745.0 }; 
                                             { SourceToTargetIndex = (55, 1) Distance = 2291.0 };
                                             { SourceToTargetIndex = (1, 55) Distance = 2291.0 }; { SourceToTargetIndex = (55, 2) Distance = 4032.0 }; 
                                             { SourceToTargetIndex = (2, 55) Distance = 4032.0 }]
                *) 


                // Take the currentGraphElement
                let (limblength,dMatrix) = graph.[n]
                // Take the currentAttachmentPoint
                let ((src,trg),distance) = attachmentsStore.[n]

        
                (* Hier kann Beispielshaft durch den Print Befehl überprüft werden; ob der  Attachmentpoint richtig übertragen wurde - Ausgabe für twentyNineMatrix

                    printfn "index: %i, src: %i, trg: %i, distance: %f"n src trg distance
                    index: 2, src: 0, trg: 1, distance: 745.000000
                    index: 3, src: 1, trg: 2, distance: 2678.000000

                *)


                (* Ab hier ist der Code noch nicht vollständig funcktionsfähig, da aktuell nur nach edgestomodify gesucht wird die genau dem src, target entsprechen, dies ist allerdings nicht ausreichend, da vorher bereits Pfade verändert wurden und im Pfad zwischen 1 und 2 z.B. schon eine internalNode eingefügt wurde
                    Problem: Es soll nodeindex 3 auf dem path zwischen nodes 1 und 2 angefügt werden, aber dieser path ist nicht verfügbar weil im vorigsen schritt schon 
                    ein artificail node (55) zwischen 1 und 2 eingefügt wurde. Deswegen müsste man resultlist jetzt untersuchen nach dem Path 1 -> 2 und die edge identifizieren an der 
                    der zusätzliche node (54) angefügt werden soll (aufgrund der distanzen 1-55: 2291 und 55-2: 4032 müsste node 54 demnach zwischen 55 und 2 in der Distanz 387 eingesetzt werden*)  

                let edgeToModify = 
                    resultList |> List.find (fun element -> 
                        element.SourceToTargetIndex = (src,trg) ||
                        element.SourceToTargetIndex = (trg,src) 
                        )

                // Hier soll der inverse Path von edge to Modify (Bsp. von 0,1 => 1,0) bestimmt und zugeordnet werden, da dieser auch aus der Liste entnommen werden muss
                let edgeToModifyInnverse = {SourceToTargetIndex = (snd (edgeToModify.SourceToTargetIndex),fst (edgeToModify.SourceToTargetIndex)); Distance = edgeToModify.Distance}
               
                (* Hier wird das im vorherigen Codeabschnitt bestimmte edgeToModify und der inverse Pfad bestimmt
                    printfn " edgetomodify: %A , edgetomodify inverse: %A"     edgeToModify edgeToModifyInnverse 
                    edgetomodify: { SourceToTargetIndex = (1, 0) Distance = 3036.0 } , edgetomodify inverse: { SourceToTargetIndex = (0, 1) Distance = 3036.0 }
                *)

                (* Hier werden die Pfade bestimmt, die in resultList hinzugefügt werden sollen und an welcher Stelle ein Knoten eingefügt werden soll
                    edgeAdditionA: Beschreibt den Pfad zwischen der Quelle und dem einzufügnden internal Node (z.B. 0 -55 ==> Distance 745 oder 55-54 ==>  distance 387 )
                    edgeAdditionB: Beschreibt den Pfad zwischen InternalNodes und dem target also von z.B. 55-1 ==> 2291 oder 54 -2 ==> 1354. 
                    Dafür muss die edgeToModiy Distanz von den einzelnen Distanzen ( Distanzen der beteiligten Teilpfade / Attachmentpoindistanz) abgezogen werden.
                    edgeAdditionC: Hier wird die aktuelle Spezies wieder zum Baum hinzugefügt im Abstand der limblength zum aktuellen InternalNode ( z.B. im ersten Schritt wird Spezies 2 wieder eingefüht)
                *)
                
                let edgeAdditionA = {SourceToTargetIndex = (src,internalNodes); Distance = distance}
                let edgeAdditionA_inverse = {SourceToTargetIndex = (internalNodes,src); Distance = distance}
                let edgeAdditionB = {SourceToTargetIndex = (internalNodes,trg); Distance = edgeToModify.Distance - distance}
                let edgeAdditionB_inverse = {SourceToTargetIndex = (trg, internalNodes); Distance = edgeToModify.Distance - distance}
                let edgeAdditionC = {SourceToTargetIndex = (internalNodes,n); Distance = limblength}
                let edgeAdditionC_inverse = {SourceToTargetIndex = (n,internalNodes); Distance = limblength}

                (* Beispielshafte Ausgabe nach dem ersten Durchlauf von edgeAddition A , edgeAddition B und edgeAddition C 
                    Edgeaddition A: { SourceToTargetIndex = (0, 55) Distance = 745.0 } 
                    edgeAdditionB: { SourceToTargetIndex = (55, 1) Distance = 2291.0 } 
                    edgeAdditionC: { SourceToTargetIndex = (55, 2) Distance = 4032.0 } 
                                                                                     *)
                // Aktualisieren von internal Nodes 

                internalNodes <- internalNodes-1

                 (* Hier wird der phylogenetische Baum  geupdated. 
                     Dafür werden die zu verändernden Pfade zunächst aus der Liste entferent und anschließend werden die vorher bestimmten Pfade der Liste hinzugefügt.
                     Zuletzt wird der übersichtlichkeithalber die Liste nach den Src sotiert und n geupdatet. Nach Beendigung der While Loop wird zuletzt die fertige resultList, 
                     welche den Baum beschreibt ausgegeben 
                *)

                let newResultList = 
                    resultList
                    |> List.where  (fun x -> x <> edgeToModify && x <> edgeToModifyInnverse)
                    |> List.append [edgeAdditionA;edgeAdditionA_inverse;edgeAdditionB;edgeAdditionB_inverse;edgeAdditionC;edgeAdditionC_inverse]
                    |>List.sortBy (fun x -> fst (x.SourceToTargetIndex))
                resultList <- newResultList
                n <- n+1
                )

        resultList
      
        
(* Beispielshafte Ausgabe zur Darstellung des Problems
printfn "resultList: %A" resultList
printfn "index: %i, src: %i, trg: %i, distance: %f"n src trg distance

resultList: [{ SourceToTargetIndex = (1, 0)
   Distance = 3036.0 }; { SourceToTargetIndex = (0, 1)
                          Distance = 3036.0 }]
index: 2, src: 0, trg: 1, distance: 745.000000
resultList: [{ SourceToTargetIndex = (0, 55)
   Distance = 745.0 }; { SourceToTargetIndex = (55, 0)
                         Distance = 745.0 }; { SourceToTargetIndex = (55, 1)
                                               Distance = 2291.0 };
 { SourceToTargetIndex = (1, 55)
   Distance = 2291.0 }; { SourceToTargetIndex = (55, 2)
                          Distance = 4032.0 }; { SourceToTargetIndex = (2, 55)
                                                 Distance = 4032.0 }]
index: 3, src: 1, trg: 2, distance: 2678.000000

In [ ]:
addPhyl twentyNineMatrix.DistanceM

resultList: [{ SourceToTargetIndex = (1, 0)
   Distance = 3036.0 }; { SourceToTargetIndex = (0, 1)
                          Distance = 3036.0 }]
index: 2, src: 0, trg: 1, distance: 745.000000
resultList: [{ SourceToTargetIndex = (0, 55)
   Distance = 745.0 }; { SourceToTargetIndex = (55, 0)
                         Distance = 745.0 }; { SourceToTargetIndex = (55, 1)
                                               Distance = 2291.0 };
 { SourceToTargetIndex = (1, 55)
   Distance = 2291.0 }; { SourceToTargetIndex = (55, 2)
                          Distance = 4032.0 }; { SourceToTargetIndex = (2, 55)
                                                 Distance = 4032.0 }]
index: 3, src: 1, trg: 2, distance: 2678.000000


Error: System.Collections.Generic.KeyNotFoundException: In der Auflistung wurde kein Index gefunden, der das Prädikat erfüllt.
   at Microsoft.FSharp.Collections.ListModule.Find[T](FSharpFunc`2 predicate, FSharpList`1 list) in D:\a\_work\1\s\src\FSharp.Core\list.fs:line 480
   at FSI_0055.addPhyl(Matrix`1 distMat)
   at <StartupCode$FSI_0056>.$FSI_0056.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [ ]:
let inputs = [1; 2; 3; 4]

inputs |> List.where (fun elm -> elm % 2 <> 0)

[ 1, 3 ] HeadOrDefault 1 TailOrNull [ 3 ] HeadOrDefault 3 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method489(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663766 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\microsoft.dotnet-interactive\1.0.451701\tools\net7.0\any\FSharp.Core.dll ModuleVersionId 6dbe05a8-5c53-690c-c522-0b7a960bbd24 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731027136472 Attributes PrivateScope, Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ParameterType T Name <null> HasDefaultValue True DefaultValue <null> RawDefaultValue <null> MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] ReturnParameter T ParameterType T Name <null> HasDefaultValue True DefaultValue <null> RawDefaultValue <null> MetadataToken 134217728 Attributes None Member T get_Head() Position -1 IsIn False IsLcid False IsOptional False IsOut False IsRetval False CustomAttributes [ ] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes (empty) Message Die Eingabeliste war leer. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136
 at lambda_method489(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 64 Tail System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method490(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663767 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\microsoft.dotnet-interactive\1.0.451701\tools\net7.0\any\FSharp.Core.dll ModuleVersionId 6dbe05a8-5c53-690c-c522-0b7a960bbd24 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a ModuleHandle System.ModuleHandle CustomAttributes [ ] IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Value 140731027136488 Attributes PrivateScope, Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList<T> Ret

In [ ]:
// graph
addPhyl myMatrixAdditive.DistanceM

resultList: [{ SourceToTargetIndex = (1, 0)
   Distance = 13.0 }; { SourceToTargetIndex = (0, 1)
                        Distance = 13.0 }]
index: 2, src: 0, trg: 1, distance: 11.000000
 edgetomodify: { SourceToTargetIndex = (1, 0)
  Distance = 13.0 }


[ { SourceToTargetIndex = (0, 5)
 Distance = 11.0 }, { SourceToTargetIndex = (5, 1)
 Distance = 2.0 }, { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (0, 5)\n Distance = 11.0 } SourceToTargetIndex (0, 5) Item1 0 Item2 5 Distance 11 TailOrNull [ { SourceToTargetIndex = (5, 1)
 Distance = 2.0 }, { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (5, 1)\n Distance = 2.0 } SourceToTargetIndex (5, 1) Item1 5 Item2 1 Distance 2 TailOrNull [ { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (5, 2)\n Distance = 10.0 } SourceToTargetIndex (5, 2) Item1 5 Item2 2 Distance 10 TailOrNull [ { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (0, 1)\n Distance = 13.0 } SourceToTargetIndex (0, 1) Item1 0 Item2 1 Distance 13 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method448(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136
 at lambda_method448(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method449(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method449(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) (empty) Head { SourceToTargetIndex = (0, 1)\n Distance = 13.0 } SourceToTargetIndex (0, 1) Item1 0 Item2 1 Distance 13 Tail [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method448(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136
 at lambda_method448(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method449(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAcc

In [ ]:
// Attachmentpoints 
addPhyl myMatrixAdditive.DistanceM

resultList: [{ SourceToTargetIndex = (1, 0)
   Distance = 13.0 }; { SourceToTargetIndex = (0, 1)
                        Distance = 13.0 }]
index: 2, src: 0, trg: 1, distance: 11.000000
 edgetomodify: { SourceToTargetIndex = (1, 0)
  Distance = 13.0 }


[ { SourceToTargetIndex = (0, 5)
 Distance = 11.0 }, { SourceToTargetIndex = (5, 1)
 Distance = 2.0 }, { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (0, 5)\n Distance = 11.0 } SourceToTargetIndex (0, 5) Item1 0 Item2 5 Distance 11 TailOrNull [ { SourceToTargetIndex = (5, 1)
 Distance = 2.0 }, { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (5, 1)\n Distance = 2.0 } SourceToTargetIndex (5, 1) Item1 5 Item2 1 Distance 2 TailOrNull [ { SourceToTargetIndex = (5, 2)
 Distance = 10.0 }, { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (5, 2)\n Distance = 10.0 } SourceToTargetIndex (5, 2) Item1 5 Item2 2 Distance 10 TailOrNull [ { SourceToTargetIndex = (0, 1)
 Distance = 13.0 } ] HeadOrDefault { SourceToTargetIndex = (0, 1)\n Distance = 13.0 } SourceToTargetIndex (0, 1) Item1 0 Item2 1 Distance 13 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method448(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136
 at lambda_method448(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method449(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method449(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) (empty) Head { SourceToTargetIndex = (0, 1)\n Distance = 13.0 } SourceToTargetIndex (0, 1) Item1 0 Item2 1 Distance 13 Tail [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method448(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Message Die Eingabeliste war leer. Data [ ] InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136
 at lambda_method448(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\r\n at lambda_method449(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAcc

In [ ]:
(addPhyl twentyNineMatrix.DistanceM).Item 2



resultList: [{ SourceToTargetIndex = (1, 0)
   Distance = 3036.0 }; { SourceToTargetIndex = (0, 1)
                          Distance = 3036.0 }]
index: 2, src: 0, trg: 1, distance: 745.000000
 edgetomodify: { SourceToTargetIndex = (1, 0)
  Distance = 3036.0 }
resultList: [{ SourceToTargetIndex = (0, 55)
   Distance = 745.0 }; { SourceToTargetIndex = (55, 1)
                         Distance = 2291.0 }; { SourceToTargetIndex = (55, 2)
                                                Distance = 4032.0 };
 { SourceToTargetIndex = (0, 1)
   Distance = 3036.0 }]
index: 3, src: 1, trg: 2, distance: 2678.000000


Error: System.Collections.Generic.KeyNotFoundException: In der Auflistung wurde kein Index gefunden, der das Prädikat erfüllt.
   at Microsoft.FSharp.Collections.ListModule.Find[T](FSharpFunc`2 predicate, FSharpList`1 list) in D:\a\_work\1\s\src\FSharp.Core\list.fs:line 480
   at FSI_0030.addPhyl(Matrix`1 distMat)
   at <StartupCode$FSI_0034>.$FSI_0034.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

#### test of some functions

Now we can construct the algorithm, which connects all prepared functions. This will be called addPhy and needs only the distanceMatrix and n as input. the distanceMatrix is called distMat and from type matrix, n is from type int and represents the species. First the distancematrix is tested for additivity if not you fail. Then you define dirst the length of matrix and assign it to speccount and you define the starting point for nodeName and assign it to internalNodelabel. For this the first internalNodelabel ist the double of matrixlength minus 3 e.g. for matrixAdditive with length 4 its 5.<br>
After definition of variables we can now call the recursive function addPhyl_helper, that need again distMat, n and internalNodelabel. This recursive function first proofs first if n is equal two, meaning that there is only two species. If this is fullfilled then you assign to distance the distance between this two matrices and add that to the dictionary. You can simply add that by using the Add function. When n is not equal two than, we directly determine the limblength and compute the d<sub>Bald</sub> Matrix. The d_Bald Matrix is now assigned to distMat and so the matrix is updated Next we need to assign the attachmentpoint to x. For this we use the function attachmentPoint, that find degenerate triples which satisfies Di,j = Di,n-1 + Dj,n-1. This formula takes dBald and n as input and wen need for x the attachmentPoint distance. Additionally we store in an Tuple (src,target) the coordinates of src and target to get attachment point of n. When this is done you call recursively the function you create and set the distanceMatrix as dBald and reduce n with one, also internalnodeLabel has to be reduced by one. By this step the graph (d_trimmed) is directly reduced and the result is assigned to the graph
Next we insert the new node on the path betwenn source and target determined during finding of attachment point. For this first the boolean array visited must be declared. At the Beginning of DFS all nodes first are false, for this reason an Array is created that has the size of the matrix and which has all values set false. Furthermore we call insert_new_node, which takes as input the actual dictionary, the src and target defined by finding the attachmentpoint and stored in a tuple, x and the actual array visited as well as internalNodelabel. Last we add leaf n-1 back to new node by creating a limb of length limbLength. for this the src node gets a new entry and also the internalNodelabel gets a new entry

In [ ]:
// Test    for y = 0 to ((snd (graph.Item n)).Column(n)).Length-1 do
//             let distance = ((snd (graph.Item n)).Column(n)).[y]
//             if distance <> 0  then
//                 if not(List.exists (fun (src,target)-> src = y) nodeList) then 
//                     resultList <- {SourceToTargetIndex = (y,internalNodes); Distance = distance}::resultList
//                     resultList <- {SourceToTargetIndex = (internalNodes,y); Distance = distance}::resultList
//                     nodeList <- (y,internalNodes)::nodeList
                    
//                 else if not (List.exists (fun (src,target)-> src = internalNodes-1) nodeList) then
//                     let remainingLength =  (((snd (graph.Item n)).Column(n)).[y]) - (((snd (graph.Item (n-1))).Column((n-1))).[y])  
//                     resultList <- {SourceToTargetIndex = (internalNodes-1,internalNodes); Distance =remainingLength}::resultList
//                     resultList <- {SourceToTargetIndex = (internalNodes,internalNodes-1); Distance = remainingLength}::resultList
//                     nodeList <- (internalNodes-1,internalNodes)::nodeList

//         internalNodes <- internalNodes-1
//         n <- n+1

//         if n = LengthOfMatrix-1 then 
//             let limb = ((fst (graph.Item n)))
//             resultList <- {SourceToTargetIndex = (n,internalNodes); Distance = limb}::resultList
//             resultList <- {SourceToTargetIndex = (internalNodes,n); Distance = limb}::resultList    

//     resultList
//     |>List.rev
//     |>List.sort





// let addPhyl<'a> (distMat:matrix) (n:int)  =
//     // testing additivity of matrix
//     testingAdditivity distMat

//     // define internalnodelabel and length of distanceMatrix, being variables of the recursive function  
//     let mutable dist_Mat = distMat
//     let mutable internalNodelabel = ((2*dist_Mat.NumRows)-3)
   
//     // called function to create graph for Additivw Phylogeny that will be recursively called
    
//     let rec addPhyl_helper(distMat:matrix) (n:int) (internalNodelabel:int)   =

//     // determine the Dictionary for n = 2, when 2*2 Matrix
//         if n = 2 then 
//             let distance = distMat.[0,1]|>int
//             let coordinatesTwoMatrix = new Dictionary<int,(int*int) list>()
//             coordinatesTwoMatrix.Add (0,[(1,distance)])
//             coordinatesTwoMatrix.Add (1,[(0,distance)])

//         // determine limblength and Dbald matrix of corresponding species and update matrix 
//         let limbLength:int = LimbLength dist_Mat n |>int
//         dist_Mat <- dBaldMatrix dist_Mat n
       
//         // find Attachmentpoint and coordinates of source and target

//         let attachment = 
//             [|for i = 0 to (n-1) do 
//                 for j = i+1 to (n-1) do
//                     if dist_Mat[i,j] = dist_Mat[i,n-1] + dist_Mat[j,n-1] then 
//                         ((i,j),dist_Mat.[i,(n-1)]|>int)
//             |]
//             |>Array.head

//         let x = snd attachment 
//         let (src,target) = fst attachment
       
//         // call recursively the function and reduce with that the graph
//         let d_trimmed = addPhyl_helper (dist_Mat) (n-1) (internalNodelabel-1)

//         // create an Array visited, which stores which node are already visited
//         let visited = Array.create (2*(dist_Mat.NumRows)) false

//         // insert new node on the path (src... dst)
//         insert_new_node (d_trimmed) (src) (target) (x) (visited) (internalNodelabel) 

//         // Add lead n-1 back 

//         d_trimmed[n-1] <- [(internalNodelabel, limbLength)]
//         d_trimmed[internalNodelabel] <- List.append d_trimmed[internalNodelabel] [(n-1, limbLength)]
      
//     d_trimmed
    


## Sammlung einzelner Teilfunktionen an einfachen Beispielen

In [ ]:
let a = 
    [
        {SourceToTargetIndex = (4,5); Distance = 8.};
        {SourceToTargetIndex = (88,5); Distance = 87.}
    ]

List.where (fun x -> fst(x.SourceToTargetIndex) <> 4) a

[ { SourceToTargetIndex = (88, 5)
 Distance = 87.0 } ] HeadOrDefault { SourceToTargetIndex = (88, 5)\n Distance = 87.0 } SourceToTargetIndex (88, 5) Item1 88 Item2 5 Distance 87 TailOrNull [ ] HeadOrDefault <null> TailOrNull <null> Head System.InvalidOperationException: Die Eingabeliste war leer.\r\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4136\r\n at lambda_method448(Closure, FSharpList`1)\r\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetV... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList<T> ReflectedType Microsoft.FSharp.Collections.FSharpList<T> MemberType Method MetadataToken 100663766 Module FSharp.Core.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\microsoft.dotnet-interactive\1.0.451701\tools\net7.0\any\FSharp.Core.dll ModuleVersionId 6dbe05a8-5c53-690c-c522-0b7a960bbd24 MetadataToken 1 ScopeName FSharp.Core.dll Name FSharp.Core.dll Assembly FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a CodeBase file:///C:/Users/Vanesssa/.nuget/packages/microsoft.dotnet-interactive/1.0.451701/tools/net7.0/any/FSharp.Core.dll FullName FSharp.Core, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a EntryPoint <null> DefinedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (2190 more) ] IsCollectible False ManifestModule FSharp.Core.dll ReflectionOnly False Location C:\Users\Vanesssa\.nuget\packages\microsoft.dotnet-interactive\1.0.451701\tools\net7.0\any\FSharp.Core.dll ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes [ Microsoft.FSharp.Core.Unit, Microsoft.FSharp.Core.SourceConstructFlags, Microsoft.FSharp.Core.CompilationRepresentationFlags, Microsoft.FSharp.Core.SealedAttribute, Microsoft.FSharp.Core.AbstractClassAttribute, Microsoft.FSharp.Core.EqualityConditionalOnAttribute, Microsoft.FSharp.Core.ComparisonConditionalOnAttribute, Microsoft.FSharp.Core.AllowNullLiteralAttribute, Microsoft.FSharp.Core.VolatileFieldAttribute, Microsoft.FSharp.Core.DefaultAugmentationAttribute, Microsoft.FSharp.Core.CLIEventAttribute, Microsoft.FSharp.Core.CLIMutableAttribute, Microsoft.FSharp.Core.AutoSerializableAttribute, Microsoft.FSharp.Core.DefaultValueAttribute, Microsoft.FSharp.Core.EntryPointAttribute, Microsoft.FSharp.Core.ReferenceEqualityAttribute, Microsoft.FSharp.Core.StructuralComparisonAttribute, Microsoft.FSharp.Core.StructuralEqualityAttribute, Microsoft.FSharp.Core.NoEqualityAttribute, Microsoft.FSharp.Core.CustomEqualityAttribute ... (214 more) ] IsFullyTrusted True CustomAttributes [ [Microsoft.FSharp.Core.FSharpInterfaceDataVersionAttribute((Int32)2, (Int32)0, (Int32)0)], [System.Runtime.Versioning.TargetFrameworkAttribute(".NETStandard,Version=v2.1", FrameworkDisplayName = ".NET Standard 2.1")], [System.Reflection.AssemblyCompanyAttribute("Microsoft Corporation")], [System.Reflection.AssemblyConfigurationAttribute("Release")], [System.Reflection.AssemblyCopyrightAttribute("© Microsoft Corporatio

In [ ]:
// succesfull
let limbLengthTest<'a> = 
    let test =
        let rawData = System.IO.File.ReadAllLines  @"C:\Users\Vanesssa\Downloads\rosalind_ba7b.txt"

        rawData
        |> Array.skip 2
        |> Array.take 27
        |> Array.map (fun x ->
            x.Split ' '
            |> Array.take 27
            |> Array.map float    
        )  
        |> matrix

    DistanceMatrix<'a>.Create [|"i";"j";"k";"l";"m";"n";"o";"p";"q";"r";"s";"t";"u";"v";"w";"x";"y";"z";"a";"b";"c";"d";"e";"f";"g"|] test

LimbLength limbLengthTest.DistanceM 5

989

In [ ]:
// Rekursion
let rec sumN x =
    if x = 0 then 
        0 
    else x + sumN(x-1)

sumN 8

36

In [ ]:
printfn"%A" (delete_species myMatrixAdditive.DistanceM 1 )

let myMatrixAnimal<'a> = 
    let distM = 
        [| 
            [|0.;3.;6.;4.|] 
            [|3.;0.;7.;5.|] 
            [|6.;7.;0.;2.|]
            [|4.;5.;2.;0.|]
        |] 
        |> matrix
        
    DistanceMatrix<'a>.Create [|"Chimp";"Human";"Seal";"Whale"|] distM

// Take the species name from it and test that
(myMatrixAnimal.Species.[0])  = "Chimp"

// Call Element in Row and Column
myMatrixAdditive.DistanceM.[0,1]

printfn "%A" myMatrixAnimal
printfn  "Limblength of Limb (species 1) is  %A" (LimblengthFormula myMatrixAnimal.DistanceM 0 3 2)

// Exchange Values in a 2D array
let a =
    [| 
        [|5.;2.;3.|]   
        [|4.;2.;1.|]   
    |]
    |>matrix


printfn "Additive Matrix is" 
printfn "%A" myMatrixAdditive.DistanceM

let all = myMatrixAdditive.DistanceM
all



           0      1      2
                          
 0 ->  0.000 21.000 22.000
 1 -> 21.000  0.000 13.000
 2 -> 22.000 13.000  0.000

Matrix of 3 rows x 3 columns
{ Species = [|"Chimp"; "Human"; "Seal"; "Whale"|]
  DistanceM =
   
          0     1     2     3
                             
 0 -> 0.000 3.000 6.000 4.000
 1 -> 3.000 0.000 7.000 5.000
 2 -> 6.000 7.000 0.000 2.000
 3 -> 4.000 5.000 2.000 0.000

Matrix of 4 rows x 4 columns}
Limblength of Limb (species 1) is  4.0
Additive Matrix is

           0      1      2      3
                                 
 0 ->  0.000 13.000 21.000 22.000
 1 -> 13.000  0.000 12.000 13.000
 2 -> 21.000 12.000  0.000 13.000
 3 -> 22.000 13.000 13.000  0.000

Matrix of 4 rows x 4 columns


[ 0, 13, 21, 22, 13, 0, 12, 13, 21, 12, 0, 13, 22, 13, 13, 0 ] Item 4 4matrix [[0;13;21;22];
[13;0;12;13];
[21;12;0;13];
[22;13;13;0]]
 ElementOps FSharp.Stats.Instances+FloatNumerics@116 NumRows 4 NumCols 4 IsDense True IsSparse False InternalSparseColumnValues System.InvalidOperationException: not a sparse matrix\r\n at FSharp.Stats.Matrix`1.get_InternalSparseColumnValues() in C:\Users\bvenn\source\repos\FSharp.Stats\src\FSharp.Stats\AlgTypes.fs:line 152\r\n at lambda_method107(Closure, DenseRepr)\r\n at Microsoft.DotNet.Interactive.Formatting.Membe... TargetSite Int32[] get_InternalSparseColumnValues() Name get_InternalSparseColumnValues DeclaringType FSharp.Stats.Matrix<T> ReflectedType FSharp.Stats.Matrix<T> MemberType Method MetadataToken 100663519 Module FSharp.Stats.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ModuleVersionId a6db3a87-bbd2-4333-294d-e59bb078f351 MetadataToken 1 ScopeName FSharp.Stats.dll Name FSharp.Stats.dll Assembly FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Users/Vanesssa/.nuget/packages/fsharp.stats/0.5.0/lib/netstandard2.0/FSharp.Stats.dll FullName FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 FSharp.Stats.IFsiFormattable 1 FSharp.Stats.IMatrixFormattable 2 FSharp.Stats.IVectorFormattable 3 FSharp.Stats.IRowVectorFormattable 4 FSharp.Stats.Interval`1[a] 5 FSharp.Stats.Interval`1+Tags[a] 6 FSharp.Stats.Interval`1+Closed[a] 7 FSharp.Stats.Interval`1+LeftOpen[a] 8 FSharp.Stats.Interval`1+RightOpen[a] 9 FSharp.Stats.Interval`1+Open[a] 10 FSharp.Stats.Interval`1+Closed@DebugTypeProxy[a] 11 FSharp.Stats.Interval`1+LeftOpen@DebugTypeProxy[a] 12 FSharp.Stats.Interval`1+RightOpen@DebugTypeProxy[a] 13 FSharp.Stats.Interval`1+Open@DebugTypeProxy[a] 14 FSharp.Stats.BigRationalLarge 15 FSharp.Stats.BigRational 16 FSharp.Stats.BigRational+Tags 17 FSharp.Stats.BigRational+Z 18 FSharp.Stats.BigRational+Q 19 FSharp.Stats.BigRational+Z@DebugTypeProxy (2084 more) IsCollectible False ManifestModule FSharp.Stats.dll MDStreamVersion 131072 FullyQualifiedName C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ModuleVersionId a6db3a87-bbd2-4333-294d-e59bb078f351 MetadataToken 1 ScopeName FSharp.Stats.dll Name FSharp.Stats.dll Assembly FSharp.Stats, Version=0.5.0.0, Culture=neutral, PublicKeyToken=null ModuleHandle System.ModuleHandle CustomAttributes [ ] ReflectionOnly False Location C:\Users\Vanesssa\.nuget\packages\fsharp.stats\0.5.0\lib\netstandard2.0\FSharp.Stats.dll ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 FSharp.Stats.IFsiFormattable 1 FSharp.Stats.IMatrixFormattable 2 FSharp.Stats.IVectorFormattable 3 FSharp.Stats.IRowVectorFormattable 4 FSharp.Stats.Interval`1[a] 5 FSharp.Stats.Interval`1+Tags[a] 6 FSharp.Stats.Interval`1+Closed[a] 7 FSharp.Stats.Interval`1+LeftOpen[a] 8 FSharp.Stats.Interval`1+RightOpen[a] 9 FSharp.Stats.Interval`1+Open[a] 10 FSharp.Stats.BigRationalLarge 11 FSharp.Stats.BigRational 12 FSharp.Stats.BigRational+Tags 13 FSharp.Stats.BigRational+Z 14 FSharp.Stats.BigRational+Q 15 FSharp.Stats.INumeric`1[T] 16 FSharp.Stats.IIntegral`1[T] 17 FSharp.Stats.IFractional`1[T] 18 FSharp.Stats.IFloating`1[T] 19 FSharp.Stats.IIEEE`1[T] (400 more) IsFullyTrusted True CustomAttributes index value 0 [Microsoft.FSharp.Core.FSharpInterfaceDataVersionAttribute((Int32)2, (Int32)0, (Int32)0)] 1 [System.Runtime.Versioning.TargetFrameworkAttribute(".NETStandard,Version=v2.0", FrameworkDisplayName = "")] 2 [System.Reflection.AssemblyCompanyAttribute("Benedikt Venn, Timo Muehlhaus, Heinrich Lukas Weil, David Zimmer, Kevin Schneider, fslab open source contributors")] 3 [System.Reflection.AssemblyConfigurationAttribute("Release")] 4 [System.Reflection.AssemblyDescriptionAttribute("F#-first linear algebra, machine 

## Algorithm translated in F# - Inser new nodes


To delete edges directly from the graph and not from matrix you have to work a little bit different. The function needs three different variables as input. Thia are d_trimmed, src and the target. d_trimmed represents an Dictionary consisting of an int value as Key and an List with tuples, describing the path where all species are visited once and highly interconected. This dictionarey is a discription of the distancematrix to reduce the graph during the recursive function. src represents the starting point and target the ending point of an path. To update the corresponding List, we use the <-  for updating this file and get no problems with the type of value

To insert internal nodes, which connect species and allow to compute back to the distancematrix, we need the function get_path, which is called by a further function get insert node. First we create the function get_path which has as input again d_trimmed the src and the target. Furthermore an Boolean array vsisited is the input. Visited represents s list, where you store for each distance, representing an path in the phylogenetic tree, if this path / edge is already visited or not. The function is divided in different steps. In General you want to compute the path by using the Depth first algorithm:<br>
<img src="DFS.png" alt="drawing" width="20%"/> <br>

First you need to define two mutable variables:
- path_list: is a list, where you store the path for a distinct source and target and, where you can add the steps in the path simply
- path_rec: variable which gets the result of the recursive called function get_path and takes first None, describing that it has actually no value 

Then the function first remembers that the src, representing the starting point, is already visited and changes this field in the Array in an true then you need to iter over the complete list in the Dictionary at an special key, here the key representing the starting node you want. This iteration can be done by using an simple for loop iterating over every tuple and doing the following statements. 
- first you need to proof if v is already visited, because v represents now the next pointr and when its visited then the path is already known (avoid doubling of pathes)
- If you have a non visited point then the next condition is to proof if v is equal to the target  then you add to the pathlist both tuples and has the path of path
- next you simply call recursive again the function by assigning the src to v and call that there is some vale, last path_list is filled by the src and w an internal node

In [ ]:
// Return a path from src to dst by DFS Depth first search.
// Path will be like [(src, w1), (v1, w2), ..., (v_k-1, w_k), (dst, 0)].
//  Define the path between two nodes bx using DFS and determining all visited nodes

let rec get_path(d_trimmed:Dictionary<int,(int*int) list>) (src: int) (target: int) (visited: Boolean array) =
    let mutable path_list: list<(int*int)> = []
    let mutable path_rec = None

    visited.[src] <- true  

    for (v, w) in d_trimmed[src] do
        if not visited[v] then 
            if v = target then( 
                path_list <- (target,0):: path_list
                path_list <- (src,w):: path_list
            )
            
            path_rec <- Some (get_path (d_trimmed) (v) (target) (visited))

            path_list <- (src,w) :: path_list
            
    path_list        

The snd formula needed is insert_new_node, which contains additionally to the variables of get_path the distance and an internalnodeLabel. distance discribes at the beginning the complete length of an path from source to target and is assigned to remainLength, that is mutable. internalnodeLabel represents the Nodename of the corresponding nodes and stores this list. Then you can create the function needed to insert the new node: 

- First we call the forward function by using the actual value
- Create mutable variables: 
    - curr gets first zero as value and is the pointer to the called node on the path
    - edgelength gets first the target node of the first tuple in the list, describing the length of the path / edge
    - remaininglength corresponds first to the complete distance between src and target 
- update values in dependence of that remainingLength is bigger than the edgeLength by using an while loop
- currNode and nextNode are defined by taking the snd value in the tuple representing the current node from interest and the next node (species)
- update field in the dictionary representing the node and how its connected , with help of List.append
- delete actual node and next node from the dictionary, representing later for every node how its connected     

 

In [ ]:
// // Insert new node named nodeName between the path (src, ..., dst).
// // The new node will be at distance 'distance' from src on the path between src and dst. --> distance represented in matrix Dbald 
// let insert_new_node (d_trimmed:Dictionary<int,(int*int) list>) (src: int) (target:int) (distance) (visited: Boolean array) (internateNodeLabel:int) =
    
//     // call function get_path and assign the result to Path, create mutable variables 
//     let path = get_path (d_trimmed) (src) (target) (visited)
    
//     let mutable curr = 0
//     let mutable edgeLength =   snd path[0]
//     let mutable remainingLength = distance

//     // update values while remainingLength is bigger than edgeLength 

//     while remainingLength >= edgeLength do
//         remainingLength <- remainingLength-edgeLength
//         curr <- curr + 1
//         edgeLength <- snd path[curr]

//     // define current node and next node by spliting it, new node is put at distance remsininglength from currNode

//     let currNode =  fst path[curr]
//     let nextNode = fst path[curr+1]

//     d_trimmed.[currNode] <- List.append d_trimmed.[currNode] [(internateNodeLabel,remainingLength)]
//     d_trimmed.[nextNode] <- List.append d_trimmed.[nextNode] [(internateNodeLabel,edgeLength-remainingLength)]
//     d_trimmed.[internateNodeLabel] <- List.append [(currNode, remainingLength)] [(nextNode, edgeLength - remainingLength)]

//     // delete edge representing curr Node as src and nextnode as target from dictionary 

//     delete_edge(d_trimmed) (currNode) (nextNode)
   
